In [1]:
import OpenDartReader
import os
import FinanceDataReader as fdr

In [2]:
api_key = '6ac6c88791643c87a1e2e0b33040e9c953a8287d'
dart = OpenDartReader(api_key)

In [3]:
stock_list_raw = fdr.StockListing('KRX').dropna()
stock_list_raw.head()


,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,70200,2,-100,-0.14,70000,70300,69500,6979146,487242293500,419078735010000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,416000,2,-13000,-3.03,418500,422000,411500,201997,83935151500,97344000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,129900,1,1900,1.48,127900,130800,127000,1846448,239306675400,94567507213500,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,724000,1,1000,0.14,718000,726000,716000,14947,10769639000,51529976000000,71174000,STK
4,005935,KR7005931001,삼성전자우,KOSPI,,56400,2,-500,-0.88,56600,56700,56100,579083,32552103000,46410809880000,822886700,STK


In [14]:
# stock_list = stock_list_raw[['Code','ISU_CD','Name','Market']]
# stock_list.head()

In [4]:
# kospi만 추출
stock_list_kospi = stock_list_raw.loc[stock_list_raw['Market'].isin(['KOSPI'])]
stock_name_list = stock_list_kospi['Name']
stock_name_list[-5:]

2757    진흥기업2우B
2759      동양2우B
2760        동양우
2761    유유제약2우B
2762     진흥기업우B
Name: Name, dtype: object

In [6]:
stock_name_list[:5]

0        삼성전자
1    LG에너지솔루션
2      SK하이닉스
3    삼성바이오로직스
4       삼성전자우
Name: Name, dtype: object

In [9]:
report = dart.finstate('삼성전자', 2022)
# report['account_nm']
# indicators = ['자산총계','부채총계','자본총계','매출액','영업이익','당기순이익','이익잉여금']
report.loc[report['fs_nm']=='연결재무제표']['account_nm']

0           유동자산
1          비유동자산
2           자산총계
3           유동부채
4          비유동부채
5           부채총계
6            자본금
7          이익잉여금
8           자본총계
9            매출액
10          영업이익
11    법인세차감전 순이익
12         당기순이익
13     당기순이익(손실)
Name: account_nm, dtype: object

In [5]:
import numpy as np
import pandas as pd

def find_financial_indicators(stock_name,year, indicators):
    report = dart.finstate(stock_name, year)
    if report is None:
        data = [[stock_name,year]+[np.nan]*len(indicators)]
        data = [[stock_name,year-1]+[np.nan]*len(indicators)]
        data = [[stock_name,year-2]+[np.nan]*len(indicators)]
    
        return pd.DataFrame(data,columns=['기업','연도']+indicators)
    else:
        report = report[report['account_nm'].isin(indicators)]
        if sum(report['fs_nm']=='연결재무제표')>0:
            report = report.loc[report['fs_nm']=='연결재무제표']
        else:
            report = report.loc[report['fs_nm']=='재무제표']
        data = []
        for y,c in zip([year,year-1,year-2],['thstrm_amount','frmtrm_amount','bfefrmtrm_amount']):
            record = [stock_name, y]
            for indic in indicators:
                if sum(report['account_nm']==indic) > 0:
                    value = report.loc[report['account_nm']==indic,c].iloc[0]
                else:
                    value = np.nan
                record.append(value)
            data.append(record)
        return pd.DataFrame(data, columns=['기업','연도']+indicators)

In [29]:
indicators = ['자산총계','부채총계','자본총계','이익잉여금','매출액','영업이익','당기순이익']
display(find_financial_indicators('삼성전자',2022,indicators)) # 2019, 2022

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000"
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000"


In [ ]:
import time 
data

In [30]:
indicators = ['자산총계','부채총계','자본총계','매출액','영업이익','당기순이익']
stock_name='삼성전자'
year=2022
report = dart.finstate(stock_name, year)
if report is None:
    data = [[stock_name,year]+[np.nan]*len(indicators)]
    data = [[stock_name,year-1]+[np.nan]*len(indicators)]
    data = [[stock_name,year-2]+[np.nan]*len(indicators)]
    data = [[stock_name,year-3]+[np.nan]*len(indicators)]
    data = [[stock_name,year-4]+[np.nan]*len(indicators)]
    result = pd.DataFrame(data,columns=['기업','연도']+indicators)
else:
    report = report[report['account_nm'].isin(indicators)]
    if sum(report['fs_nm']=='연결재무제표')>0:
        report = report.loc[report['fs_nm']=='연결재무제표']
    else:
        report = report.loc[report['fs_nm']=='재무제표']
    data = []
    for y,c in zip([year,year-1,year-2],['thstrm_amount','frmtrm_amount','bfefrmtrm_amount']):
        record = [stock_name, y]
        for indic in indicators:
            if sum(report['account_nm']==indic) > 0:
                value = report.loc[report['account_nm']==indic,c].iloc[0]
            else:
                value = np.nan
            record.append(value)
        data.append(record)
    result = pd.DataFrame(data, columns=['기업','연도']+indicators)
display(result)

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2022,"448,424,507,000,000","93,674,903,000,000","354,749,604,000,000","302,231,360,000,000","43,376,630,000,000","55,654,077,000,000"
1,삼성전자,2021,"426,621,158,000,000","121,721,227,000,000","304,899,931,000,000","279,604,799,000,000","51,633,856,000,000","39,907,450,000,000"
2,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"


In [16]:
result = dart.finstate('LG전자',2022,'11011')
result = result.loc[result['fs_nm']=='연결재무제표']
result = result.loc[result['account_nm'].isin(['당기순이익','영업이익','매출액'])]
result = result.loc[:,['account_nm','fs_nm','sj_nm','thstrm_dt','thstrm_amount']]
display(result)

,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
9,매출액,연결재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"83,467,318,000,000"
10,영업이익,연결재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"3,550,972,000,000"
12,당기순이익,연결재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"1,863,123,000,000"
